In [1]:
import requests as req
from bs4 import BeautifulSoup as bs
import re
import js2xml
import pandas as pd

In [2]:
url = 'https://socialblade.com/youtube/user/getmovies/monthly'
graph = "graph-youtube-monthly-subscribers-container"
#graph = "graph-youtube-monthly-vidviews-container"

In [3]:
def ParseOne(url, graph):
    from urllib.request import Request, urlopen
    rq = Request(url , headers={'User-Agent': 'Mozilla/5.0'})
    urlop = urlopen(rq)
    if (urlop.status == 200):
        raw = urlop.read()
        bspage = bs(raw, 'html.parser')
        print(url)
    else:
        print(urlop.status)
    
    script = bspage.find_all('script', {'type': 'text/javascript'}, text=re.compile(graph))[0].contents[0]
    parsed = js2xml.parse(script)
    bsjs = bs(js2xml.pretty_print(parsed), 'xml')
    bschart = bsjs.find('string',  text=re.compile(graph)).parent
    xchart = bschart.find('property', {'name': 'data'}).find('array')
    
    xdatapoints = xchart.find_all('array')
    dplist = []
    for i in xdatapoints: 
        dp = []
        for j in i.find_all('number'):
            dp.append(int(j['value']))
        dplist.append(dp)
        
    df = pd.DataFrame(dplist, columns = ['tstp', 'value'])
    df['dtime']=pd.to_datetime(df['tstp'], unit='ms')
    df['url']=url
    df['graph']=graph
    df = df[['url','graph','dtime', 'value']] 
    
    return(df)

In [4]:
dfout = ParseOne(url, graph)
dfout

https://socialblade.com/youtube/user/getmovies/monthly


,url,graph,dtime,value
0,https://socialblade.com/youtube/user/getmovies...,graph-youtube-monthly-subscribers-container,2021-07-01 04:00:00,35800000
1,https://socialblade.com/youtube/user/getmovies...,graph-youtube-monthly-subscribers-container,2021-06-01 04:00:00,35600000
2,https://socialblade.com/youtube/user/getmovies...,graph-youtube-monthly-subscribers-container,2021-05-01 04:00:00,35400000
3,https://socialblade.com/youtube/user/getmovies...,graph-youtube-monthly-subscribers-container,2021-04-01 04:00:00,35000000
4,https://socialblade.com/youtube/user/getmovies...,graph-youtube-monthly-subscribers-container,2021-03-01 05:00:00,34600000
5,https://socialblade.com/youtube/user/getmovies...,graph-youtube-monthly-subscribers-container,2021-02-01 05:00:00,34200000
6,https://socialblade.com/youtube/user/getmovies...,graph-youtube-monthly-subscribers-container,2021-01-01 05:00:00,33800000
7,https://socialblade.com/youtube/user/getmovies...,graph-youtube-monthly-subscribers-container,2020-12-01 05:00:00,33200000
8,https://socialblade.com/youtube/user/getmovies...,graph-youtube-monthly-subscribers-container,2020-11-01 04:00:00,32900000
9,https://socialblade.com/youtube/user/getmovies...,graph-youtube-monthly-subscribers-container,2020-10-01 04:00:00,32300000


In [ ]:
dfout.to_excel('./parsed.xlsx')